# Not used

In [ ]:
def assign_random_plus(adata, K):
    gene_factor_list = []
    n_spots, n_genes = adata.X.shape
    X = adata.X
    wt = np.zeros((K, n_genes))
    dt = np.zeros ((n_spots, K))
    all_n_umis = adata.X.sum(axis=1)
    for n_UMIs in all_n_umis:
        factors = np.random.randint(low = 0,
                                    high = K,
                                    size = int(n_UMIs))
        gene_factor_list.append(factors.tolist())
    return gene_factor_list

In [ ]:
def count_gene_factor_old(gene_factor, K):
    gene_factor_count = np.zeros((K, len(gene_factor[1,:])))
    for factor in range(K):
        count = np.count_nonzero(gene_factor == factor, axis = 0)
        gene_factor_count[factor] = count
    return gene_factor_count

In [ ]:
wt = count_gene_factor(gene_factor, 3)

In [ ]:
# How many times (across all spots) is each and every gene (columns) assigned to each factor (rows).
wt

In [ ]:
def count_spot_factor_old(gene_factor, K):
    spot_factor_count = np.zeros((len(gene_factor), K))
    for factor in range(K):
        count = np.count_nonzero(gene_factor == factor, axis = 1)
        spot_factor_count[:,factor] = count
    return spot_factor_count

In [ ]:
dt = count_spot_factor(gene_factor, 3)

In [ ]:
# For each spot (rows), how many genes are assigned to each and every factor (columns).
dt

In [ ]:
# dimensions n_spots rows and K columns. 
def count_spot_factor(gene_factor, K):
    n_spots, n_genes = adata.X.shape
    spot_factor_count = np.zeros((n_spots, K))
    for spot in range(n_spots):
        spot_list = gene_factor[spot].tolist()
        for factor in range(K):
            spot_factor_count[spot, factor] = spot_list.count(factor)
    return spot_factor_count

In [ ]:
dt = count_spot_factor(random_factors, 3)

In [ ]:
dt

In [ ]:
def count_gene_factor(adata, gene_factor, K): # Ge ids som input
    n_spots, n_genes = adata.X.shape
    gene_factor_count = np.zeros((K, n_genes))
    X = adata.X
    for spot in range(n_spots):
        spot_list = gene_factor[spot].tolist()
        start = 0
        end = 0
        for gene in range(n_genes):
            end += int(X[spot, gene])
            for factor in range(K):
                gene_factor_count[factor, gene] += spot_list[start: end].count(factor)
            start = end
    return gene_factor_count

In [ ]:
wt = count_gene_factor(adata, random_factors, 3)

In [ ]:
wt

# Used code starts here

## Load data and packages

In [1]:
import random
import numpy as np
import scanpy as sc
import anndata as an
import scipy.stats as st
from scipy.spatial import KDTree
import math
from numba import njit
import numba

In [2]:
import numba_scipy

In [4]:
def LoadData(path):
    adata = sc.read_visium(path)
    adata.var_names_make_unique()
    return adata

In [5]:
path ="/Users/juliafoyer/Documents/Skolarbete/Masters_thesis/human_breast_cancer_ST_data"
adata = LoadData(path)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [6]:
X = np.array(adata.X.todense())

In [ ]:
adata.X = X

In [ ]:
X.shape

In [ ]:
X[4]

In [ ]:
adata.X[4] = adata.X[4] / 2

In [ ]:
adata.X[4]

In [7]:
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata)

In [10]:
adata = sc.read_visium(path)

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [11]:
def prepare_data(adata : an.AnnData, select_hvg : bool = True)-> an.AnnData:
    adata.var_names_make_unique()
    X = np.array(adata.X.todense())
    adata.X = X
    if select_hvg:
        sc.pp.log1p(adata)
        sc.pp.highly_variable_genes(adata, n_top_genes = 1000)
#        adata = adata[:,adata.var.highly_variable_genes.values]
        adata = adata[:,adata.var.highly_variable.values]
        adata.X = (np.exp(adata.X) -1).round(0)
    return adata

In [8]:
adata = adata[:,adata.var.highly_variable.values]

In [9]:
# Another option is to save genes, reload data and then filter
adata.X = (np.exp(adata.X) -1).round(0)

TypeError: loop of ufunc does not support argument 0 of type SparseCSRView which has no callable exp method

In [ ]:
X = adata.X

In [ ]:
spots = adata.obs_names
genes = adata.var_names
n_spots = len(adata.obs_names)
n_genes = len(adata.var_names)

In [ ]:
edge_influence = 5

In [ ]:
def distance_fun2(p,q):
    #Bhattacharyya distance
    return - np.log(np.sqrt(p*q).sum())

In [ ]:
%%time
distance_fun2(theta[0], theta[1])

In [ ]:
%%time
distance_fun(theta[0], theta[1])

In [12]:
# 2x speed
@njit(parallel=False)
def distance_fun(p,q):
    #Bhattacharyya distance
    return - np.log(np.sqrt(p*q).sum())

In [13]:
# njit makes it worse
#@njit(parallel=False)
def log_potential(edge_influence, current_theta, neighbour_theta):
    return edge_influence * distance_fun(current_theta, neighbour_theta)

In [14]:
def build_nbrhd(adata,
            n_neighbours):
    """ add short description here
    
    Parameters:
    ----------
    """
    kd = KDTree(adata.obsm["spatial"])
    dist,indx = kd.query(adata.obsm["spatial"], k = n_neighbours) # why not +1?
    no_nbr = np.isinf(dist)
    dist[no_nbr] = 0
    dist[~no_nbr] = 1
    indx[no_nbr] = -1
    return dist, indx

In [15]:
dist, indx = build_nbrhd(adata, 6)

In [ ]:
%%time
sample_theta(0, theta, dt, indx, dist, edge_influence, 3)

In [16]:
def get_nz(dt):
    nz = dt.sum(axis=0)
    return nz

In [ ]:
nz = get_nz(dt)

In [ ]:
@njit(parallel=False)
def draw_new_factor(umi_factors,
                    dt,
                    wt,
                    nz,
                    d,
                    index,
                    w):
    z = umi_factors[d][index]
    dt[d, z] = dt[d, z] - 1
    wt[z, w] = wt[z, w] - 1
    nz[z] = nz[z] - 1
    pz = np.divide(np.multiply(dt[d, :], wt[:, w]), nz)
    z = np.random.multinomial(1, (pz / pz.sum())).argmax()
    umi_factors[d][index] = z 
    dt[d, z] = dt[d, z] + 1
    wt[z, w] = wt[z, w] + 1
    nz[z] = nz[z] + 1

In [ ]:
@njit(parallel=False)
def draw_new_factor(sub_list,
                    dt,
                    wt,
                    nz,
                    d,
                    index,
                    w):
    
    z = sub_list[index]
    dt[d, z] = dt[d, z] - 1
    wt[z, w] = wt[z, w] - 1
    nz[z] = nz[z] - 1
    pz = np.divide(np.multiply(dt[d, :], wt[:, w]), nz)
    z = np.random.multinomial(1, (pz / pz.sum())).argmax()
    sub_list[index] = z 
    dt[d, z] = dt[d, z] + 1
    wt[z, w] = wt[z, w] + 1
    nz[z] = nz[z] + 1

In [ ]:
@njit(parallel=False)
def draw_new_factor(sub_list,
                    dt,
                    wt,
                    nz,
                    d,
                    index,
                    w):
    
    z = sub_list[index]
    dt[d, z] -=  1
    wt[z, w] -= 1
    nz[z] -= 1
    pz = np.divide(np.multiply(dt[d, :], wt[:, w]), nz)
    z = np.random.multinomial(1, (pz / pz.sum())).argmax()
    sub_list[index] = z 
    dt[d, z] += 1
    wt[z, w] += 1
    nz[z] += 1

In [ ]:
%%time
draw_new_factor(umi_factors[0], dt, wt, nz, 0, 0, 0)

In [ ]:
%%time
draw_new_factor(umi_factors, dt, wt, nz, 0, 0, 0)

In [ ]:
type(umi_factors[0])

In [ ]:
nz

In [ ]:
%%time
sample_theta(0, theta, dt, indx, dist, edge_influence, 3)

In [ ]:
type(np.random.dirichlet(dt[0,:]))

In [ ]:
@njit(parallel=False)
def log_pdf_dirichlet(x,alpha):
    l_beta_top = 0
    l_beta_bot = 0
    frac = 0
    for _a,_x in zip(alpha,x):
        l_beta_top += m.log(m.gamma(_a))
        l_beta_bot += _a
        frac += (_a - 1)*m.log(_x)
    l_beta_bot = np.log(m.gamma(l_beta_bot))
    return frac + l_beta_bot - l_beta_top

@njit(parallel=False)
def dirichlet_sample(alpha):
    n = len(alpha)
    y = np.zeros(n)
    ys = 0
    for i in range(n):
        y[i] = np.random.gamma(shape=alpha[i],scale=1)
        ys += y[i]
    y = y / ys
    return y

In [ ]:
import math as m
spot = 0
alpha = dt[spot,:]
#alpha = np.array([[253., 238., 234.]])
print(alpha)
print(type(alpha))
theta_mat = np.zeros((1, 3))
theta_mat[0,:] = theta[spot,:]
#x = np.array([[0.34820951, 0.33147761, 0.32031288]])
print(theta_mat[0,:])
print(type(theta_mat[0,:]))
old_log_proposal = log_pdf_dirichlet(x, alpha)
print(old_log_proposal)

In [ ]:
new_theta = dirichlet_sample(alpha)
print(new_theta)
new_log_proposal = log_pdf_dirichlet(new_theta, alpha)
print(new_log_proposal)

In [ ]:
def sample_theta(spot, theta, dt, indx, dist, edge_influence, K, n_iter = 10):
        
#    proposal_dist = st.dirichlet(dt[spot,:])
    alpha = dt[spot,:]
    
    theta_mat = np.zeros((n_iter+1, K))
        
    theta_mat[0,:] = theta[spot,:]
#    old_log_proposal = proposal_dist.logpdf(theta_mat[0,:])
    old_log_proposal = log_pdf_dirichlet(theta_mat[0,:], alpha)
    old_log_target = log_potential(edge_influence, theta_mat[0,:], theta[indx[spot,:]])
    old_log_target *= dist[spot,:] # Keep only those that are real neighbours. * 0 cancels false neighbours.
    old_log_target = old_log_target.sum()
    
    for it in range(1,n_iter+1):
#        new_theta = proposal_dist.rvs()[0]
        new_theta = dirichlet_sample(alpha)
#        new_log_proposal = proposal_dist.logpdf(new_theta)
        new_log_proposal = log_pdf_dirichlet(new_theta, alpha)
        new_log_target = log_potential(edge_influence, new_theta, theta[indx[spot,:]])
        new_log_target *= dist[spot,:]
        new_log_target = new_log_target.sum()
                    
        log_u = new_log_target + old_log_proposal - old_log_target - new_log_proposal
        u = np.exp(log_u) 
        a = min(u,1)
        
        if np.random.random() < a:
            theta_mat[it,:] = new_theta
            old_log_target = new_log_target
            old_log_proposal = new_log_proposal
        else:
            theta_mat[it,:] = theta_mat[it-1,:]
                    
        theta[spot,:] = theta_mat.mean(axis=0)

In [ ]:
@njit(parallel=False)
def sample_theta(spot, theta, dt, indx, dist, edge_influence, K, n_iter = 10):
        
    n_topics = dt.shape[0]
    theta_mat = np.zeros((n_iter+1, K))
    
    proposal_dist = np.zeros(n_topics)
    proposal_dist = np.random.dirichlet(dt[spot,:])

        
    theta_mat[0,:] = theta[spot,:]
    old_log_proposal = proposal_dist.logpdf(theta_mat[0,:])
    old_log_target = log_potential(edge_influence, theta_mat[0,:], theta[indx[spot,:]])
    old_log_target *= dist[spot,:] # Keep only those that are real neighbours. * 0 cancels false neighbours.
    old_log_target = old_log_target.sum()
    
    for it in range(1,n_iter+1):
        new_theta = proposal_dist.rvs()[0] # what is the 0?
        new_log_proposal = proposal_dist.logpdf(new_theta)
        new_log_target = log_potential(edge_influence, new_theta, theta[indx[spot,:]])
        new_log_target = new_log_target * dist[spot,:]
        new_log_target = new_log_target.sum()
                    
        log_u = new_log_target + old_log_proposal - old_log_target - new_log_proposal
        u = np.exp(log_u) 
        a = min(u,1)
        
        if np.random.random() < a:
            theta_mat[it,:] = new_theta
            old_log_target = new_log_target
            old_log_proposal = new_log_proposal
        else:
            theta_mat[it,:] = theta_mat[it-1,:]
                    
        theta[spot,:] = theta_mat.mean(axis=0)

In [30]:
1.5 * 400 / 60

10.0

## Assign random variables

In [18]:
def assign_random(adata, K):
    umi_factor_list = []
    all_n_umis = adata.X.sum(axis=1)
    for n_UMIs in all_n_umis:
        factors = np.random.randint(low = 0,
                                    high = K,
                                    size = int(n_UMIs))
        umi_factor_list.append(factors)
    umi_factor_list = np.array(umi_factor_list, dtype=object)
    return umi_factor_list

In [19]:
umi_factors = assign_random(adata, 3)

In [ ]:
def first_edge_influence(lambda0, theta, indx_sel, K):
    DB = lambda p, q, X: - math.log(Bhatt_coeff(p, q, X))
    edge_influence = []
    n_spots = len(indx_sel)
    X = K
    for spot in range(n_spots):
        p = theta[0]
        influences = []
        neighbours = indx_sel[spot][1:]
        for neighbour in neighbours:
            q = theta[neighbour]
            influences.append(np.random.exponential(lambda0 + DB(p, q, X)))
        edge_influence.append(influences)
    return np.array(edge_influence, dtype=object)

In [ ]:
edge_influence = first_edge_influence(lambda0, theta, indx_sel, 3)

## Get IDs and count matrices

In [22]:
def get_ids_dt_wt(adata, umi_factors, K):
    n_spots, n_genes = adata.X.shape
    X = adata.X
    ids = []
    dt = np.zeros((n_spots, K))
    wt = np.zeros((K, n_genes))
    for spot in range(n_spots):
        ids_spot = []
        spot_list = umi_factors[spot].tolist()
        start = 0
        end = 0
        for gene in range(n_genes):
            n_umis = int(X[spot, gene])
            ids_spot += [gene] * n_umis
            end += n_umis
            for factor in range(K):
                wt[factor, gene] += spot_list[start: end].count(factor)
            start = end
        ids.append(ids_spot)
        for factor in range(K):
            dt[spot, factor] = spot_list.count(factor)
    return ids, dt, wt

In [27]:
def get_ids_dt_wt(adata,
                  umi_factors,
                  K,
                  alpha = 0.1,
                  beta = 0.1):
    """ add short description here
    
    Parameters:
    ----------
    adata : an.AnnData
        anndata object to study
    umi_factors. List[np.ndarray]
        List of arrays. Element j in list i represents
        topic that word j belongs to in spot i
    K: int
        number of topics

    Returns:
    --------
    
    """
    
    n_spots, n_genes = adata.X.shape
    X = adata.X
    ids = []
    dt = np.zeros((n_spots, K)) + alpha
    wt = np.zeros((K, n_genes)) + beta
    
    for spot in range(n_spots):
        ids_spot = []
        spot_list = umi_factors[spot].tolist()
        start = 0
        end = 0
        for gene in range(n_genes):
            n_umis = int(X[spot, gene])
            ids_spot += [gene] * n_umis
            end += n_umis
            for factor in range(K):
                wt[factor, gene] += spot_list[start: end].count(factor)
            start = end
        ids.append(ids_spot)
        for factor in range(K):
            dt[spot, factor] += spot_list.count(factor)
    return ids, dt, wt

In [23]:
umi_factors

array([array([0, 1, 2, ..., 0, 0, 2]), array([1, 2, 1, ..., 2, 2, 2]),
       array([2, 0, 1, ..., 0, 1, 1]), ...,
       array([0, 1, 2, ..., 0, 1, 1]), array([0, 1, 0, ..., 0, 0, 0]),
       array([2, 0, 2, ..., 0, 1, 1])], dtype=object)

In [28]:
ids, dt, wt = get_ids_dt_wt(adata, umi_factors, 3)

KeyboardInterrupt: 

In [ ]:
assert(np.all(np.sum(dt,axis=1) == np.sum(X,axis=1))), "total UMI's in spot and dt rowsums must be equal"

## Functions for calculating theta and phi

In [ ]:
def get_theta(dt):
    n_spots, K = dt.shape
    theta = np.zeros((n_spots, K))
    for spot in range(n_spots):
        for factor in range(K):
            theta[spot, factor] = dt[spot, factor]/np.sum(dt, axis=1)[spot]
    return theta

In [ ]:
theta = get_theta(dt)

In [ ]:
def get_phi(wt): # I don't use this...
    K = 3
    phi = np.zeros((K, n_genes))
    for factor in range(K):
        for gene in range(n_genes):
            phi[factor, gene] = gene_factor_count[factor, gene]/np.sum(gene_factor_count, axis=1)[factor]
    return phi

In [ ]:
def get_phi(wt):
    phi = gene_factor_count/gene_factor_count.sum(axis=1, keepdims=True)
    return phi

In [ ]:
phi = get_phi(wt)

## Get neighbours and graph edges

In [ ]:
def findKNN(adata, n_neighbours):
    kd = KDTree(adata.obsm["spatial"])
    dist,indx = kd.query(adata.obsm["spatial"], k = n_neighbours + 1)
    return dist, indx

In [ ]:
dist, indx = findKNN(adata, 6)

In [ ]:
dist

In [ ]:
u,c = np.unique(indx[:,-1],return_counts = True)

In [ ]:
indx

In [ ]:
def remove_false_neighbours(dist, indx, eps = 300):
    dist_sel = []
    indx_sel = []
    for i in range(len(dist)):
        dist_sel.append([])
        indx_sel.append([])
        for j in range(len(dist[i])):
            if dist[i][j] < eps:
                dist_sel[i].append(dist[i][j])
                indx_sel[i].append(indx[i][j])
        dist_sel[i] = np.array(dist_sel[i])
        indx_sel[i] = np.array(indx_sel[i])
    return np.array(dist_sel, dtype=object), np.array(indx_sel, dtype=object)

In [ ]:
dist_sel, indx_sel = remove_false_neighbours(dist, indx)

In [ ]:
len(indx_sel)

In [ ]:
indx_sel

In [ ]:
edges = 0
for i in indx:
    edges += len(i)
print(edges)

In [ ]:
def get_E(indx_sel):
    edges = 0
    for i in indx_sel:
        edges += len(i) - 1
    return edges

In [ ]:
E = get_E(indx_sel)

In [ ]:
E

## Functions for Gibbs sampling of variables

In [ ]:
def sample_lambda(E, edge_influence, lambda_a = 0.01, lambda_b = 0.01):
    sum_edge_influences = 0
    for edge in edge_influence:
        sum_edge_influences += sum(edge)
    shape = lambda_a + E
    rate = 1 / (lambda_b + sum_edge_influences)
    lambda_parameter = np.random.gamma(shape, rate, 1)
    return lambda_parameter

In [ ]:
def sample_inv_edge_influence(edge_influence, lambda_parameter, theta, indx_sel, K):
    n_spots = len(indx_sel)
    X = K
    for spot in range(n_spots):
        p = theta[spot]
        index = 1
        n_neighbours = len(indx_sel[spot][1:])
        for index in range(n_neighbours):
            neighbour = indx_sel[spot][index+1]
            q = theta[neighbour]
            edge_influence[spot][index] = np.random.exponential(lambda_parameter - math.log(Bhatt_coeff(p, q, X)))

In [ ]:
def draw_new_factor(gene_factor, ids, spot, w, dt, wt, theta, K, beta = 0.1):
    factor0 = gene_factor[spot][w]
    gene_id = ids[spot][w]
    dt[spot, factor0] -= 1
    wt[factor0, gene_id] -= 1
    left = theta[spot,]
    right = (wt[:,gene_id] + beta) / (wt.sum(axis=0)[gene_id] + K * beta)
    pvals = left*right
    pvals = pvals / pvals.sum()
    factor1 = np.random.choice(K, p = pvals)
    dt[spot, factor1] +=  1
    wt[factor1, gene_id] += 1
    gene_factor[spot][w] = factor1

In [ ]:
def Gibbs(gene_factor, ids, dt, wt, theta, K, iterations, edge_influence, lambda_parameter, indx_sel, E, adata, beta = 0.1):
    for it in range(iterations):
        for spot in range(len(gene_factor)):
            for w in range(len(gene_factor[spot])):
                draw_new_factor(gene_factor, ids, spot, w, dt, wt, theta, K)
        sample_edge_influence(edge_influence, lambda_parameter, theta, indx_sel, K)
        lambda_parameter = sample_lambda(E, edge_influence, lambda_a = 0.01, lambda_b = 0.01)
        metr_hast(adata, theta, indx_sel, edge_influence, lambda_parameter, 3)

In [ ]:
%%time
# Before metropolis-Hastings was included.
Gibbs(random_factors, ids, dt, wt, theta, 3, 1, edge_influence, lambda_parameter, indx_sel, E)

In [ ]:
%%time
# After metropolis-Hastings included. No difference. Draw_new_factor takes all the time.
Gibbs(random_factors, ids, dt, wt, theta, 3, 1, edge_influence, lambda_parameter, indx_sel, E, adata)

## Metropolis-Hastings sampling of theta

In [ ]:
@njit(parallel=False)
def Bhatt_coeff(p, q, X):
    BC = 0
    for x in range(X):
        BC += math.sqrt(p[x]*q[x])
    return BC

In [ ]:
def metr_hast(adata, theta, indx_sel, edge_influence, lambda_parameter, K, iterations = 10):
    
    # Prepare proposal distribution    
    dist_g = lambda x: np.random.dirichlet(x)
    
    # Acceptance probability
    DB = lambda p, q, X: - math.log(Bhatt_coeff(p, q, X))
    X = K
    
    # Metropolis-Hastings sampling
    n_spots, n_genes = adata.X.shape    
    for spot in range(n_spots):
        log_a_prob0 = 0 # Cannot use from last Gibbs iteration, because there will be new edge_infl and lambdas.
        log_a_prob1 = 0
        p0 = theta[spot]
        p1 = dist_g(np.sum(theta, axis=0)) 
        neighbours = indx_sel[spot][1:]
        index = 0
        for neighbour in neighbours:
            q = theta[neighbour]
            log_edge_potential0 = - DB(p0,q,X) * edge_influence[spot][index] # = - DB(p,q,X) / inv_ldd ? Eller?
            log_edge_potential1 = - DB(p1,q,X) * edge_influence[spot][index]
            log_a_prob0 += log_edge_potential0
            log_a_prob1 += log_edge_potential1
            index += 1
        a_prob = min(1, np.exp(log_a_prob1 - log_a_prob0)) 
        print(a_prob) # This is really large. Maybe that's normal because everything is better than random?
        if np.random.random() < a_prob:
            p0 = p1
        print(theta[spot])
        theta[spot] = p0
        print(theta[spot])
        
        
##        neighbours = indx[spot][1:]
  #      for neighbour in neighbours:
 #           q = theta[neighbour]
 #           inv_ldd = 1 / edge_influence[neighbour]
 #           log_edge_potential = - DB(p,q,X) / inv_ldd
 #           log_a_prob +=log_edge_potential
 #       a_prob = min(1, np.exp(log_a_prob))
 #           
 #       theta1 = dist_g(theta0)
            
            
  #          a = edge_potentials1 / edge_potentials0
  #          a = min(1, a)
  #          if np.random.random() < a:
  #              theta0 = theta1
  #      theta[spot] = theta0

In [ ]:
a_prob, lambda_parameter = acc_prob(0, indx, theta, 3, lambda_parameter0)

In [ ]:
np.random.dirichlet((1279, 1296, 1223))

In [ ]:
np.random.dirichlet((0.3, 0.3, 0.3))

In [ ]:
metr_hast(adata, theta, indx_sel, edge_influence, lambda_parameter, 3, iterations = 10)

In [ ]:
def acc_prob(spot, indx, theta, K, inv_ldd,lambda_parameter):
    DB = lambda p, q, X: - math.log(Bhatt_coeff(p, q, X))
#    get_lambda_parameter = lambda shape, scale : np.random.gamma(shape, scale, 1)
    get_edge_influence = lambda p, q, X : 1 / (np.random.exponential(lambda_parameter + DB(p, q, X))) # "measure of strength"
#    get_edge_potential = lambda p, q, X : math.exp(- get_edge_influence(p, q, X) * DB(p, q, X))

    p = theta[spot]
    
    a_prob = 1
    neighbours = indx[spot][1:]
    for neighbour in neighbours:
        q = theta[neighbour]
        X = K
       # edge_potential = math.exp(- 1 / (np.random.exponential(lambda_parameter + DB(p, q, X))) * DB(p, q, X))
        #edge_potential = math.exp(-DB(p, q, X) / inv_ldd[neighbour])
        #print(edge_potential)
        log_edge_potential = - DB(p,q,X) / inv_ldd[neighbour]
        #a_prob *= edge_potential
        log_a_prob +=log_edge_potential
    a_prob = min(1, np.exp(a_prob))
    
    # Get lambda parameter for next round
    lambda_a = 0.01
    lambda_b = 0.01
    E = np.sum(indx) / 2
    shape = lambda_a + E
    scale = lambda_b + get_edge_influence(p, q, X) * E
    lambda_parameter = np.random.gamma(shape, 1/scale, 1)
    
    return a_prob, lambda_parameter